# 1. Overview: Preprocessing Notebook
This notebook preprocesses election contribution data and shapefiles from https://open-data.bouldercolorado.gov/, and creates preliminary visualizations with Uber's H3 API https://h3geo.org/docs/. Radar's geocoding API is used to gather lat/lon pairs from record addresses https://radar.com/product/geocoding-api. Finally, functions are defined throughout the notebook to make it clearer as to what's being used, when, and why.

Listed below are a few of the core problems tackled throughout this notebook:
1) "City" feature cleaning
    - Donations to elections can span the entire country; there are no restrictions on who can and cannot give money. Because of this, we see many donations that fall outside of Boulder's city limits. Since this analysis is only interested in donations from Boulder proper, it's important to filter donations.
    - Human error is rampant in this dataset, leading to records with misspellings of "Boulder", empty values for State and Zip columns, and more. The city spelling in particular takes quite a bit of work to reconstruct. A class named *Matcher* was created to identify all Boulder entries regardless of spelling. There are more advanced string matching options and packages, but this was a nice opportunity to demonstrate custom code.
2) Record identification
    - Even after proper city names were recovered, the problem of missing and misspelled state and zip columns remained. Layered logic repaired these records leading to a largely clean dataset. The output is not perfect, but it's "good enough" for the next step. 
3) Geocoding
    - Correct addresses aren't very helpful without lat/lon pairs when mapping data. As a solution, Custom functions are built to call Radar's geocoding API which returns a formatted address and a lat/lon pair. Note that performance is poor since batch requests are not allowed with a free account. In a production-setting, this code would be altered significantly to improve overall speed.
    
    
Finally, a handful of H3 visualizations are included to view contributions across the city. H3 is an open-source package with several advantages, many of which are discussed at length on their blog and in documentation. I prefer H3 to geohash or S2, because hexagons (and their associated "rings" which surround them) are roughly a circle. This has advantages when creating radius-based features such as contributions_within_15_miles.$^{*}$

<font size="2">
$^{*}$ H3-specific notebooks in my repo go into greater detail
</font>

___

## 1.1 Package Imports & Dataload

In [40]:
import pandas as pd
import math
import numpy as np
import json
import requests
import time
import re
import geopandas as gpd
from shapely.ops import transform
from shapely import Point, Polygon
import h3

In [231]:
# if we were using AWS 
# import awswrangler as wr 
# df = wr.s3.read_csv(S3_PATH)

In [3]:
# import csv
df = pd.read_csv('election_amounts.csv')
# view all cities
df.City.unique()

array(['Boulder', 'Longmont', 'BOULDER', 'Chicago', 'Ward', 'boulder',
       'Denver', nan, 'Olney', 'Ann Arbor', 'Nederalnd', 'Superior',
       'Lafayette', 'Nederland', 'Littleton', 'B', 'Arvada', 'Louisville',
       'Bouler', 'Champaign', 'BOulder', 'Erie', 'Santa Cruz', 'Lyons',
       'Fort Collins', 'Westminster', 'Austin', 'Orinda', 'Anahein',
       'Bolder', 'Asheville', 'Charlottesville', 'Nederland ', 'Oakland',
       'Anaheim', 'Boulder ', 'Firestone', 'Greenwood Village',
       'Broomfield', '50', 'Greenwich', 'Greenville', 'Washington',
       'Coc Cob', 'Portland', 'Westport', 'Rye', 'Burdett', 'Valparaiso',
       'Albion', 'Romulus', 'Englewood', 'la', 'montclair', 'maimi beach',
       'longmont', 'miami', 'Westminister', 'Columbia', 'Maricopa',
       'Gainsville', 'Trumansburg', 'Niwot', 'Vilas', 'West Chester',
       'online', 'gypsum', 'chandler', 'sausalito',
       'Dulwich Hill NSW, Au', 'Dulwich, NSW Austral', 'Dulwich Hill NSW',
       'woodland park', 

We only want to consider "Boulder" along with its associated misspellings. In order to do this, we'll implement fuzzy matching. A custom matching method is created below because of the interest in matching a static dataset against a static word. A productionized solution for many different city names would require a more sophisticated approach.

___

## 1.2 City Cleaning

#### Idea: Match = Minimum Number of *Matching* Characters Met
- The class below is involved, but at its core it defines a match when 
    - **1)** >75% of the characters are matched and 
    - **2)** when the word we are checking is within 1 character (in terms of length) from the base word. 
- self.duplicate is created for letters_in_common and threshold_met
    - If we have duplicate letters in a word, the more performant set logic breaks down. In this case we'll fall back to list logic.

In [4]:
class Matcher:
    '''
    We define a class with a base word and an identifier "duplicate".
    If duplicate is true, meaning there are duplicate letters within a word,
    then we use the list logic in letters_in_common. If duplicate is false,
    then we use the set logic which has greater performance. By instantiating the
    class, we automatically create the duplicate variable opposed to checking
    for duplicates every time within letters_in_common.
    '''
    def __init__(self, base_word):
        self.base_word = base_word.lower()
        self.base_word_length = len(base_word)
        self.duplicate = False if len(set(base_word))==len(list(base_word)) else True
        
    def stringify(self, to_check):
        '''
        Preprocess to_check to address situations like:
        1. 'boulder-CO'
        2. '   boulder '
        3. nan
        4. <int>/<float>
        '''
            # First, check if to_check is a string and not NaN
        if pd.isna(to_check):
            return ''  # Return an empty string for NaN values
        elif isinstance(to_check, str):
            stripped = to_check.strip()
            if stripped:  # Check if the string is not empty after stripping
                return stripped.split()[0].lower()
            else:
                return '' 
        else:
            return ''  
        
    def letters_in_common(self, to_check):
        to_check = self.stringify(to_check)
        if self.duplicate:
            return len([x for x in to_check if x in self.base_word])
        else:
            return len(set(self.base_word).intersection(set(to_check)))
    
    def threshold_met(self, to_check, threshold=0.75):
        to_check = self.stringify(to_check)
        if self.duplicate:
            return True if len([x for x in to_check if x in self.base_word])/self.base_word_length > threshold else False
        else:
            return True if len(set(self.base_word).intersection(set(to_check)))/self.base_word_length > threshold else False
        
    def length_req_met(self, to_check, diff=1):
        to_check = self.stringify(to_check)
        return True if int(abs(self.base_word_length-len(to_check))) <= int(diff) else False
        
    def common_word(self, to_check):
        to_check = self.stringify(to_check)
        return True if (self.length_req_met(to_check) and self.threshold_met(to_check)) else False

In [5]:
m = Matcher('Boulder')
df.loc[df['City'].apply(lambda x: m.common_word(x))].City.unique()

array(['Boulder', 'BOULDER', 'boulder', 'Bouler', 'BOulder', 'Bolder',
       'Boulder ', 'coulder', 'Bouldera', 'Bouder', 'Boudler', 'Bouldr',
       'buolder', 'Boul;der', ' Boulder ', 'Boulde', 'bulder', 'Bouilder',
       'Boulder County', 'oulder', 'Noulder', ' Boulder', 'Boukder',
       'Bouldwer', 'Bouldewr', 'oOulder', 'bOUlder', 'Bouolder', 'bolder',
       'bouulder', 'Boulder  ', 'bouder'], dtype=object)

Seeing 'Boulder County', it's clear that checking only for city name matches still leaves errors. There are three other avenues we can expand into for greater accuracy (we will only look at 1 and 2, since 3 is out of scope for this exercise):

1) Check for State
    - Rather simple; all states must = CO
2) Check for Zip
    - Given a list of all Boulder ZIPs 
3) Match street name
    - It should be possible to take every street name and, with a google maps API or something similar, see of the address exists within Boulder. This presents a host of other problems since street addresses are highly repetitive across the country and are subject to change. This option won't be investigated.
    
Given that we will be plotting lat/lon data generated from (street, city, state, zip) tuples, it's important that we have all of these features present and largely accurate. It seems unlikely that an individual would leave out both State and ZIP from their donation, so we'll add binary indicators for each of these columns. We'll also add a binary indicator for City. With these three binary indicators (and null street names dropped), logic can be implemented to keep all correct columns for Boulder-proper donations.

At least two of State/City/Zip must match to be eligible for the final dataset. But first, we'll replace all city matches with "Boulder" and make sure that all state pairs for "Boulder" are "CO". Since Boulder is not a unique city name, we expose ourselves to associating (Boulder, other state) pairs with (Boulder, CO), however the likelihood of this is low and will be handled later when gathering lat/lons from addresses. Once this step is complete, indicator columns will be generated. From this point forward, all changes will be made to a copy of df.

___

## 1.3 City/State/Zip Filtering

In [6]:
# copy of df
cp = df.copy()

# city replacement
cp['City'] = cp['City'].apply(lambda x: 'Boulder' if m.common_word(x) else x)
cp['city_match'] = cp['City'].apply(lambda x: 1 if m.common_word(x) else 0)

# state replacement
## this creates cases where we can have (NaN, CO) city/state tuples. If zip==Boulder zip, we'll re-populate NaN cp.City with "Boulder"
cp['State'] = cp.apply(lambda x: 'CO' if x.city_match==1 else x.State, axis=1)
cp['state_match'] = cp['State'].apply(lambda x: 1 if isinstance(x, str) and str.strip(x).split()[0].lower()=='co' else 0)

# zip replacement
zips = ['80301', '80302', '80303', '80304', '80305', '80503', '80306', '80307', '80308', '80309', '80310', '80314']
cp['Zip'] = cp['Zip'].apply(lambda x: str.strip(x).split()[0] if isinstance(x, str) and str.strip(x).split()[0] in zips else x)
cp['zip_match'] = cp['Zip'].apply(lambda x: 1 if isinstance(x, str) and str.strip(x).split()[0] in zips else 0)

# match count
cp['match_count'] = cp.city_match + cp.state_match + cp.zip_match

In [7]:
# check for uniqueness given "Boulder"
cp.loc[cp['City'].apply(lambda x: m.common_word(x))].City.unique()

array(['Boulder'], dtype=object)

#### Restrict to 2 matches
- output all unique (City, State, Zip) tuples

In [9]:
# restrict to 2 matches
cp = cp[cp.match_count>=2]
# output tuples
set(cp[['City', 'State', 'Zip']].itertuples(index=False))

{Pandas(City=', Boulder', State='CO', Zip='80302'),
 Pandas(City='10/1/2015', State='CO', Zip='80304'),
 Pandas(City='50', State='CO', Zip='80302'),
 Pandas(City='80303', State='CO', Zip='80303'),
 Pandas(City='9/10/2015', State='CO', Zip='80301'),
 Pandas(City='B', State='CO', Zip='80302'),
 Pandas(City='B', State='CO', Zip='80303'),
 Pandas(City='B', State='CO', Zip='80304'),
 Pandas(City='B', State='CO', Zip='80305'),
 Pandas(City='Bluff', State='CO', Zip='80304'),
 Pandas(City='Boulder', State='CO', Zip='100'),
 Pandas(City='Boulder', State='CO', Zip='480305'),
 Pandas(City='Boulder', State='CO', Zip='50'),
 Pandas(City='Boulder', State='CO', Zip='8/3/2005'),
 Pandas(City='Boulder', State='CO', Zip='80-305'),
 Pandas(City='Boulder', State='CO', Zip='80023'),
 Pandas(City='Boulder', State='CO', Zip='80027'),
 Pandas(City='Boulder', State='CO', Zip='80034'),
 Pandas(City='Boulder', State='CO', Zip='80202'),
 Pandas(City='Boulder', State='CO', Zip='8030-4'),
 Pandas(City='Boulder', St

#### Further Cleaning
- 80503 is primarily Longmont, but does overlap with Boulder
    - if 80503 and ~Boulder then we'll drop

In [10]:
# drop longmont 
cp = cp[~((cp.Zip.str.split().str[0]=='80503') & ~(cp.City.str.split().str[0]=='Boulder'))]

In [11]:
set(cp[['City', 'State', 'Zip']].itertuples(index=False))

{Pandas(City=', Boulder', State='CO', Zip='80302'),
 Pandas(City='10/1/2015', State='CO', Zip='80304'),
 Pandas(City='50', State='CO', Zip='80302'),
 Pandas(City='80303', State='CO', Zip='80303'),
 Pandas(City='9/10/2015', State='CO', Zip='80301'),
 Pandas(City='B', State='CO', Zip='80302'),
 Pandas(City='B', State='CO', Zip='80303'),
 Pandas(City='B', State='CO', Zip='80304'),
 Pandas(City='B', State='CO', Zip='80305'),
 Pandas(City='Bluff', State='CO', Zip='80304'),
 Pandas(City='Boulder', State='CO', Zip='100'),
 Pandas(City='Boulder', State='CO', Zip='480305'),
 Pandas(City='Boulder', State='CO', Zip='50'),
 Pandas(City='Boulder', State='CO', Zip='8/3/2005'),
 Pandas(City='Boulder', State='CO', Zip='80-305'),
 Pandas(City='Boulder', State='CO', Zip='80023'),
 Pandas(City='Boulder', State='CO', Zip='80027'),
 Pandas(City='Boulder', State='CO', Zip='80034'),
 Pandas(City='Boulder', State='CO', Zip='80202'),
 Pandas(City='Boulder', State='CO', Zip='8030-4'),
 Pandas(City='Boulder', St

In [12]:
# drop addresses 
cp = cp.dropna(subset=['Street']).reset_index(drop=True)

There are still some errors in the dataset, but it's much cleaner than when we started and largely looks good enough to plug into Radar's geocoder. NULL addresses are dropped since we can't recover lat/lons without these present.

___

## 1.4 Final Preprocessing

#### Radar Geocode API
- The functions below all fold into *fill_address* which is used to populate address, lat, and lon. 
- If we were productionizing a workflow, many of these hard-coded values would be removed and replaced with arguments

In [14]:
def custom_filter(value):
    return value not in [None, 'nan', np.nan]

def geolocate(pre_address):
    headers = {
        'Authorization': auth,
    }
    
    query=f'{pre_address}&layers=address&limit=1&near=40.00533,-105.25555'
    url=f'https://api.radar.io/v1/geocode/forward?query={query}'   
    response = requests.get(url=url, headers=headers)
    
    if (response.json()['meta']['code']!=200) or (not response.json()['addresses']):
        return None
    else:
        return response.json()

def clean_street(row):
    # Define our conditions
    keywords = ['#', 'unit', 'apt']
    pattern = re.compile(r'(' + '|'.join(re.escape(keyword) for keyword in keywords) + ').*$', re.IGNORECASE)
    
    return re.sub(pattern, '', row).strip()
    
# fill address based on available information; return full address and lat/lon
def fill_address(row):
    if row.name % 500 == 0:
        print(f'ROWS COMPLETE: {row.name} | TIME: {time.time()-start}')
        
    # Clean Street
    row['Street'] = clean_street(str(row['Street']))
    
    # Build a partial address as a string
    pre_address = (' '.join(filter(custom_filter, [str(row['Street']), str(row['City']), str(row['State']), str(row['Zip'])]))).replace(' ', '+')
    
    # Attempt to geocode the partial address
    location = geolocate(pre_address)
    if location:
        return pd.Series([location['addresses'][0]['formattedAddress'],
                          location['addresses'][0]['latitude'],
                          location['addresses'][0]['longitude']])
    else:
        return pd.Series([None, None, None])


# load auth 
f = open('auth.txt', 'r')
auth = f.read()

# Apply the function to the DataFrame
start = time.time()
cp[['Full Address', 'Latitude', 'Longitude']] = cp.apply(fill_address, axis=1)
print(f'ROWS COMPLETE: {len(cp)} | TIME: {time.time()-start}')

ROWS COMPLETE: 0 | TIME: 0.009482145309448242
ROWS COMPLETE: 500 | TIME: 153.06374716758728
ROWS COMPLETE: 1000 | TIME: 313.90986013412476
ROWS COMPLETE: 1500 | TIME: 476.0237412452698
ROWS COMPLETE: 2000 | TIME: 646.268764257431
ROWS COMPLETE: 2500 | TIME: 786.6126773357391
ROWS COMPLETE: 3000 | TIME: 923.8980710506439
ROWS COMPLETE: 3500 | TIME: 1075.9698433876038
ROWS COMPLETE: 4000 | TIME: 1215.3671431541443
ROWS COMPLETE: 4500 | TIME: 1345.1409602165222
ROWS COMPLETE: 5000 | TIME: 1518.2834422588348
ROWS COMPLETE: 5500 | TIME: 1646.832602262497
ROWS COMPLETE: 6000 | TIME: 1787.4929921627045
ROWS COMPLETE: 6500 | TIME: 1926.911333322525
ROWS COMPLETE: 7000 | TIME: 2062.0605742931366
ROWS COMPLETE: 7500 | TIME: 2204.4894511699677
ROWS COMPLETE: 8000 | TIME: 2331.265037059784
ROWS COMPLETE: 8042 | TIME: 2345.323178052902


In [15]:
# save since process is time consuming
cp.to_csv('radar_data.csv')

#### Spatial Join Geometries to City Limits Shapefile
- Boulder proper city limits are loaded in an plotted
- Geopandas spatial join ensures all lat/lon pairs fall within Boulder proper

In [232]:
def load_convert_shapefile(shapefile, geometry_col='geometry', epsg=4269, conv_epsg=4326):
    '''
    This function takes in a shapefile and returns a GeoDataFrame with the proper coordinates.
    '''
    # read in shapefile
    gdf = gpd.read_file(shapefile)
    
    # if the CRS is not set, set it to NAD83 (EPSG:4269) which is a typical default if missing (per google)
    if gdf.crs is None:
        gdf.set_crs(epsg=epsg, inplace=True)
        print(f'EPSG manually set to: {epsg}')
    else: 
        epsg = str(gdf.crs).split(':')[1]
        print(f'EPSG found: {epsg}')

    # convert to WGS84
    gdf = gdf.to_crs(epsg=conv_epsg)
    print(f'EPSG converted: {conv_epsg}')
    
    return gdf

def get_xy(x, y, z):
    '''
    Takes in multipolygon Z and removes z-coord
    '''
    return x, y

In [233]:
# shapefile path
shapefile = 'boulder/City_of_Boulder_City_Limits.shp'
gdf = load_convert_shapefile(shapefile)
bgdf = gpd.GeoDataFrame(geometry=[gdf.geometry.unary_union], crs=4326)
bgdf['geometry'] = bgdf['geometry'].apply(lambda x: transform(get_xy, x))
bgdf.explore()

EPSG found: 4326
EPSG converted: 4326


In [234]:
# create Point geometry
cp['geometry'] = cp.apply(lambda x: Point(x.Longitude, x.Latitude), axis=1)

# create final dataframe
cpf = cp.copy()
cpf = gpd.GeoDataFrame(cpf, geometry='geometry', crs=4326)

# inner spatial join with boulder polygon
cpf = cpf.sjoin(bgdf, how='inner').drop(columns='index_right')

___

## 1.5 Preliminary H3 Output
- Resolution 9 hexagons are used which correspond to a diameter of 1/4mi
- generate_shapefile_h3_cells() is based on a function I created for use in production pipelines
    - h3.polygon_to_cells() will only keep hexagons with majority overlap on the Polygon in question. This results in many edge hexagons being removed from the dataset. The "exterior hexagons" portion pulls these edge hexagons back into the dataset.

In [235]:
def generate_shapefile_h3_cells(gdf, resolution):
    '''
    A GeoDataFrame is inserted with polygons spanning some geography. We fill that geography with H3 
    hexagons at a given resolution.
    '''
    
    h3_cell_ids = []
    polyct = 0
    multipolyct = 0
    
    for cell in gdf.geometry: 

        # check to see if the geometry is a polygon or multipolygon
        if cell.geom_type == 'Polygon': 
            # list to gather coordinates and exterior hexagons
            coords = []
            outer_hex_ids = []

            # iterate over coordinates in each polygon
            for coord in cell.exterior.coords: 
                # reverse lat/lon order for H3
                coords.append(tuple((coord[1], coord[0])))
                
                # exterior hexagons
                outer_hex_ids.append(h3.latlng_to_cell(coord[1], coord[0], resolution))

            # get interior and partial exterior hexagons
            inner_hex_ids = list(h3.polygon_to_cells(h3.Polygon(coords), resolution))

            # add all hexagons to hex_id list
            h3_cell_ids += list(set(outer_hex_ids)) + inner_hex_ids

            # add to poly counter 
            polyct += 1

        else: 
                
            # iterate over every polygon within the multipolygon
            for idx, poly in enumerate(cell.geoms):
                # list to gather coordinates and exterior hexagons
                coords = []
                outer_hex_ids = []
                
                # iterate over coordinates in each polygon
                for coord in poly.exterior.coords:
                    # reverse lat/lon order for H3
                    coords.append(tuple((coord[1], coord[0])))
                    
                    # exterior hexagons
                    outer_hex_ids.append(h3.latlng_to_cell(coord[1], coord[0], resolution))

                # get interior and partial exterior hexagons
                inner_hex_ids = list(h3.polygon_to_cells(h3.Polygon(coords), resolution))
                
                # add all hexagons to hex_id list
                h3_cell_ids += list(set(outer_hex_ids)) + inner_hex_ids

            # add to multipoly counter
            multipolyct += 1 

    print(f'------RESOLUTION {resolution}------')
    print(f'H3 Hexagons Generated: {len(set(h3_cell_ids))}')
    print(f'MultiPolygons: {multipolyct}')
    print(f'Polygons: {polyct}')
    print('\n')

    return list(set(h3_cell_ids))

In [236]:
cells = generate_shapefile_h3_cells(bgdf, 9)

------RESOLUTION 9------
H3 Hexagons Generated: 776
MultiPolygons: 1
Polygons: 0




In [237]:
# hexagons lat/lng must be flipped for accurate plotting
def flip(x,y):
    return y,x
geoms = [transform(flip, Polygon(h3.cell_to_boundary(cell))) for cell in cells]

In [238]:
hexes = gpd.GeoDataFrame(pd.DataFrame(list(zip(cells, geoms)), columns=['hex_id', 'geometry']), geometry='geometry', crs=4326)
hexes.explore()

#### Capping and Grouping
- Huge donations make it hard to visualize contributions. To control for this, a capped contribution variable is created which limits the contribution size to 100. In subsequent modelling steps, we'll control for outliers with greater precision.

In [239]:
# join hex data
cpf = cpf.sjoin(hexes, how='left')

# turn contributions to float
cpf['Contribution'] = cpf.Contribution.apply(lambda x: float(re.sub('[$(),]', '', x.strip())))

# check the 90th percentile and ceiling the data in a separate column
# this helps control for outliers
cap = cpf.Contribution.quantile(0.90)
print(cap)
cpf['Contribution_Cap'] = cpf.Contribution.apply(lambda x: min(x, cap))

# save data to csv
cpf.to_csv('processed_data.csv', index=False)

# create a groupby for plotting
cpfg = cpf.groupby('hex_id').agg(contribution_total=('Contribution', 'sum'),
                                 capped_total=('Contribution_Cap', 'sum'),
                                 unique_contributions=('ObjectId', 'count')).reset_index()

# add average contribution amounts
cpfg['per_contribution_amt'] = round(cpfg.contribution_total/cpfg.unique_contributions,2)
cpfg['per_contribution_amt_cap'] = round(cpfg.capped_total/cpfg.unique_contributions,2)

# add a geometry column
cpfg['geometry'] = cpfg.hex_id.apply(lambda x: transform(flip, Polygon(h3.cell_to_boundary(x))))
cpfg = gpd.GeoDataFrame(cpfg, geometry='geometry', crs=4326)

100.0


#### Unique Contributions

In [240]:
cpfg.explore('unique_contributions', cmap='RdYlGn')

#### Capped Contribution Total

In [241]:
cpfg.explore('capped_total', cmap='RdYlGn')

#### Capped Average Contribution

In [242]:
cpfg.explore('per_contribution_amt_cap', cmap='RdYlGn')